# Web Research Sub-Agent Test Notebook

This notebook tests the web research sub-agent, which is responsible for:
- **Finding Information**: Searching for relevant, reliable sources
- **Gathering Data**: Collecting facts, statistics, and quotes
- **Documenting Sources**: Keeping detailed records of where information came from
- **Assessing Initial Quality**: Noting if sources seem reliable or questionable

The main agent uses this sub-agent to:
- Research why stock prices are moving
- Analyze recent price action, volume, and volatility
- Investigate company-specific news, earnings, and guidance
- Gather sector trends, competitor info, and macro factors
- Collect datasets for the analysis agent to process

## Prerequisites

This notebook connects to the LangGraph server, which provides:
- **Cloud-hosted store** - Persistent memory across sessions (same as LangSmith Studio)
- **Cloud-hosted checkpointer** - Conversation state persistence

### Setup Steps

1. **Start the LangGraph server** (from the `deep-agent/` directory):
   ```bash
   cd deep-agent
   langgraph dev
   ```

2. **Wait for the server** to be ready at `http://localhost:2024`

3. **Run this notebook** - it connects to the same agent as LangSmith Studio

### Why This Architecture?

When you run `langgraph dev`, LangGraph connects to LangSmith's cloud infrastructure.
This means:
- Agent state persists across sessions
- No local PostgreSQL database required
- Same agent instance as LangSmith Studio

## Setup

In [ ]:
# Ensure scratchpad folders exist and are empty
from pathlib import Path
import shutil

scratchpad = Path("../scratchpad")
for folder in ["data", "images", "notes", "plots", "reports"]:
    path = scratchpad / folder
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True)
    
print("Scratchpad folders ready (data, images, notes, plots, reports)")

In [ ]:

from langgraph_sdk import get_sync_client
from dotenv import load_dotenv

load_dotenv()

# Connect to local LangGraph server (must have `langgraph dev` running)
LANGGRAPH_URL = "http://localhost:2024"

try:
    client = get_sync_client(url=LANGGRAPH_URL)
    assistants = client.assistants.search(limit=10)
    if not assistants:
        raise RuntimeError("No assistants registered. Run `langgraph dev` from the deep-agent/ directory and try again.")

    ASSISTANT_ID = assistants[0]["assistant_id"]

    print(f"Connected to LangGraph server at {LANGGRAPH_URL}")
    print("Available assistants:")
    for a in assistants:
        marker = " (selected)" if a["assistant_id"] == ASSISTANT_ID else ""
        print(f"  - {a['assistant_id']}: {a.get('name', 'unnamed')}{marker}")
    print(f"Using assistant_id: {ASSISTANT_ID}")
except Exception as e:
    print(f"Could not connect to LangGraph server at {LANGGRAPH_URL}")
    print(f"Error: {e}")
    print("Make sure to run 'langgraph dev' from the deep-agent/ directory first.")
    raise SystemExit(1)


## Helper Function to Test the Agent

In [ ]:
import time
from IPython.display import display, Markdown


def truncate(text, limit=2000):
    return text[:limit] + "\n..." if len(text) > limit else text


def test_web_research_agent(message: str, thread_id: str = None):
    """Run the web research agent via LangGraph SDK and display all intermediate steps."""
    thread_id = thread_id or f"test-{int(time.time())}"

    try:
        client.threads.get(thread_id)
    except Exception:
        client.threads.create(thread_id=thread_id)

    display(Markdown(f"## Task\n```\n{message.strip()}\n```\n---"))

    final_response = None

    for chunk in client.runs.stream(
        thread_id=thread_id,
        assistant_id=ASSISTANT_ID,
        input={"messages": [{"role": "user", "content": message}]},
        stream_mode="updates",
    ):
        if chunk.event == "updates":
            for node_name, node_output in chunk.data.items():
                messages = node_output.get("messages", [])
                for msg in messages:
                    msg_type = msg.get("type")

                    if msg_type == "ai" and msg.get("tool_calls"):
                        for tc in msg["tool_calls"]:
                            name = tc.get("name")
                            args = tc.get("args", {})
                            display(Markdown(f"### Tool Call: `{name}`\n```json\n{truncate(str(args), 500)}\n```"))

                    elif msg_type == "tool":
                        content = msg.get("content", "")
                        display(Markdown(f"### Tool Response\n```\n{truncate(content)}\n```\n---"))

                    elif msg_type == "ai" and msg.get("content") and not msg.get("tool_calls"):
                        final_response = msg["content"]
                        display(Markdown(f"## Response\n{final_response}"))

    return final_response


---
# Example 1: Single Stock Research (Simple)

**Context**: A trader wants to understand why a specific stock moved significantly.

**Sub-agent role**: Research recent news and events related to the stock and provide cited findings.

In [ ]:
# Example 1: Single stock research
example_1_message = """Research why Microsoft (MSFT) stock has been moving recently.

Focus on:
- Recent news or announcements
- Any earnings or guidance updates
- AI-related developments

Provide your findings with proper source citations.
"""

import time
response_1 = test_web_research_agent(example_1_message, thread_id=f"example-1-{int(time.time())}")

---
# Example 2: Sector Analysis (Medium)

**Context**: A portfolio manager wants to understand sector-wide trends affecting their holdings.

**Sub-agent role**: Research the semiconductor sector, identify key trends, and gather relevant data.

In [ ]:
# Example 2: Sector analysis
example_2_message = """Research the current state of the semiconductor sector.

I need to understand:
1. How major semiconductor stocks have performed recently (NVDA, AMD, INTC, AVGO)
2. Key trends driving the sector (AI demand, supply chain, geopolitics)
3. Recent analyst sentiment and price target changes
4. Any upcoming catalysts or risks

Save any relevant data you find to the scratchpad/data/ folder.
Provide properly cited sources for all claims.
"""

response_2 = test_web_research_agent(example_2_message, thread_id=f"example-2-{int(time.time())}")

---
# Example 3: Comprehensive Event Research (Complex)

**Context**: A risk manager needs to understand the market implications of a major economic event.

**Sub-agent role**: Research across multiple asset classes and provide a comprehensive view with data.

In [ ]:
# Example 3: Comprehensive event research
example_3_message = """Research the market impact of the most recent Federal Reserve interest rate decision.

I need comprehensive research covering:

1. **The Decision Itself**
   - What was the decision (rate change, guidance)
   - How did it compare to market expectations
   - Key quotes from Fed Chair's statement

2. **Market Reactions Across Asset Classes**
   - Equity indices (S&P 500, Nasdaq, small caps)
   - Bond yields (2Y, 10Y treasuries)
   - Dollar index and gold

3. **Sector-Specific Impacts**
   - Which sectors benefited vs suffered
   - Rate-sensitive stocks (banks, REITs, utilities)
   - Growth vs value performance

4. **Forward-Looking Implications**
   - Market pricing for future rate decisions
   - Analyst commentary on the path forward
   - Risks and uncertainties

Save any relevant data or notes to the scratchpad folders.
Every claim must have a source citation.
"""

response_3 = test_web_research_agent(example_3_message, thread_id=f"example-3-{int(time.time())}")

---
# Verify Output Files

Check what data and notes were saved to the scratchpad directories.

In [ ]:
import os
from pathlib import Path

scratchpad_dir = Path("../scratchpad")

# Check data directory
data_dir = scratchpad_dir / "data"
if data_dir.exists():
    data_files = list(data_dir.glob("*"))
    if data_files:
        print(f"Found {len(data_files)} files in scratchpad/data/:")
        for f in sorted(data_files, key=lambda x: x.stat().st_mtime, reverse=True):
            print(f"  - {f.name}")
    else:
        print("scratchpad/data/ is empty")

print()

# Check notes directory
notes_dir = scratchpad_dir / "notes"
if notes_dir.exists():
    notes_files = list(notes_dir.glob("*"))
    if notes_files:
        print(f"Found {len(notes_files)} files in scratchpad/notes/:")
        for f in sorted(notes_files, key=lambda x: x.stat().st_mtime, reverse=True):
            print(f"  - {f.name}")
    else:
        print("scratchpad/notes/ is empty")

---
# Notes

## Expected Outputs

For each example, the web research agent should provide:

### 1. Summary of Findings
- Each statement with inline citation: [Source Name](URL)

### 2. Key Facts with Citations
- Format: "Fact statement [Source Name](URL)"
- No uncited claims

### 3. Citations List
- Numbered list of all sources used
- Brief description of what info came from each source

### 4. Gaps in Research
- What couldn't be found or verified

## Data Output

When the agent finds useful data, it should save to:
- `scratchpad/data/` - CSV files, datasets for analysis
- `scratchpad/notes/` - Detailed research notes

## Best Practices Being Tested

- Multiple search queries to triangulate information
- Noting the date of sources (recency matters)
- Distinguishing between news, opinion, and research
- Flagging unreliable sources

## Integration with Main Agent

In production, the main agent would:
1. Assign research tasks to `web-research-agent`
2. Receive findings with citations
3. Send findings to `credibility-agent` for verification
4. Pass verified data to `analysis-agent` for processing
5. Compile everything into final reports